In [18]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_deepseek import ChatDeepSeek
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnablePassthrough
from langchain_community.utils.math import cosine_similarity
import numpy as np

RunnableBranch: 基于条件选择哪个分支继续运行 + 默认分支

In [19]:
llm = ChatDeepSeek(
    model="deepseek-chat", 
    temperature=0, 
    api_key=os.getenv("DEEPSEEK_API_KEY")
    )

sichuan_route_prompt = "你是一位处理川菜的专家。用户的问题是关于麻辣、辛香、重口味的菜肴，\
    例如水煮鱼、麻婆豆腐、鱼香肉丝、宫保鸡丁、花椒、海椒等。"
cantonese_route_prompt = "你是一位处理粤菜的专家。用户的问题是关于清淡、鲜美、原汁原味的菜肴，\
    例如白切鸡、老火靓汤、虾饺、云吞面等。"

route_prompts = [sichuan_route_prompt, cantonese_route_prompt]
route_names = ['川菜', '粤菜']

embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh-v1.5")
routes_embeddings = embedding_model.embed_documents(route_prompts)
print(f"已定义 {len(route_names)} 个路由: {', '.join(route_names)}")


已定义 2 个路由: 川菜, 粤菜


## 设置不同菜系的处理链

In [20]:
sichuan_prompt = PromptTemplate.from_template(
    "你是一位川菜大厨。请用正宗的川菜做法，回答关于 【{query}】的问题"
)

sichuan_chain = sichuan_prompt | llm | StrOutputParser()

cantonese_chain = (
    PromptTemplate.from_template(
        "你是一位粤菜大厨。请用经典的粤菜做法，回答关于【{query}】的问题。")
    | llm
    | StrOutputParser()
)

route_map = {
    "川菜": sichuan_chain,
    "粤菜": cantonese_chain
}

# 定义备用通用链
general_prompt = PromptTemplate.from_template(
    "你是一个美食助手。请回答关于【{query}】的问题。"
)
general_chain = general_prompt | llm | StrOutputParser()


## 创建路由函数

In [24]:
# 3. 创建路由函数
def route_by_embedding(info):
    # 对用户查询进行嵌入
    query_embedding = embedding_model.embed_query(info['query'])

    # 计算与各路由提示的余弦相似度
    similarity_scores = cosine_similarity([query_embedding], routes_embeddings)[0]
    print(f"相似度分数 {similarity_scores}")

    # 找到最相似的路由
    index = np.argmax(similarity_scores)
    max_name = route_names[index]

    print(f"路由决策: 检测到问题与“{max_name}”最相似。argmax下标: {index}")

    # 获取对应的处理链
    chain = route_map[max_name]

    # 直接调用选中的链并返回结果
    return chain.invoke(info)

# 创建完整的路由链
full_router_chain = RunnableLambda(route_by_embedding)
    

## 运行演示查询

In [25]:
demo_questions = [
    "水煮鱼怎么做才嫩？",        # 应该路由到川菜
    "如何做一碗清淡的云吞面？",    # 应该路由到粤菜
    "麻婆豆腐的核心调料是什么？",  # 应该路由到川菜
]

for i, ques in enumerate(demo_questions, 1):
    ques += "回答少于100字"
    query = {"query": ques}
    print(f"\n--- 问题 {i}: {ques} ---")

    try:
        # 直接执行完整链路
        answer = full_router_chain.invoke(query)
        print(f"回答: {answer}")

    except Exception as e:
        print(f"执行错误: {e}")


--- 问题 1: 水煮鱼怎么做才嫩？回答少于100字 ---
相似度分数 [0.51655393 0.49110705]
路由决策: 检测到问题与“川菜”最相似。argmax下标: 0
回答: 鱼片嫩滑关键三点：  
1. **选鱼**：用黑鱼或鲈鱼，斜刀片薄片，厚度约2毫米。  
2. **码味**：鱼片加盐、料酒、葱姜水抓黏，再放蛋清和红薯淀粉锁住水分。  
3. **火候**：汤微沸时滑入鱼片，保持小火浸煮约90秒，鱼片变白立即捞出，靠余温烫熟。  
（汤底需提前用豆瓣酱、花椒、辣椒炒香，加水煮出味后捞出料渣再煮鱼）

--- 问题 2: 如何做一碗清淡的云吞面？回答少于100字 ---
相似度分数 [0.49414788 0.59434825]
路由决策: 检测到问题与“粤菜”最相似。argmax下标: 1
回答: **汤底：** 老母鸡、猪骨、大地鱼干、虾头慢火煲4小时，滤清备用。  
**云吞：** 鲜虾仁三分肥猪肉七分，加蛋清、生粉轻拌，薄皮包裹。  
**面：** 竹升生面焯水10秒，入碗。  
**组合：** 云吞沸水煮90秒，与面同置，浇热汤，撒韭黄段即成。汤清面爽，鲜而不浊。

--- 问题 3: 麻婆豆腐的核心调料是什么？回答少于100字 ---
相似度分数 [0.62877276 0.4560471 ]
路由决策: 检测到问题与“川菜”最相似。argmax下标: 0
回答: 麻婆豆腐核心调料为：郫县豆瓣酱（提香辣）、汉源花椒（出麻香）、豆豉（增醇厚）、辣椒面（助红亮）、蒜苗（增鲜解腻）。正宗做法需“麻、辣、烫、香、酥、嫩、鲜、活”八字兼备。
